# Import Libs

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.optim import Adam, lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import models, transforms
from torch.utils.tensorboard import SummaryWriter

from PIL import Image
import pandas as pd
import numpy  as np
import time
import os

# Configuration

In [2]:
classes = [
    "angry", 
    "disgust", 
    "fear", 
    "happy", 
    "sad", 
    "surprise", 
    "neutral"
]

In [3]:
class Config:
    # dataset
    TRAIN_DS_PATH = './dataset/train.csv'
    VAL_DS_PATH = './dataset/val.csv'
    TEST_DS_PATH = './dataset/test.csv'
    
    # images dir
    TRAIN_IMG_DIR = './dataset/train/'
    VAL_IMG_DIR  = './dataset/val/'
    TEST_IMG_DIR  = './dataset/finaltest/'
    
    # training hyperparams
    EPOCHS = 10
    LR = 1e-4
    BATCH_SIZE = 8
    NUM_WORKERS = 0
    SHUFFLE = True
    
    # saved model path
    MODEL_DIR = './model/'

In [4]:
cfg = Config()

In [5]:
# tensorboard writer
writer = SummaryWriter("runs")

# Dataset

In [6]:
# reference: https://github.com/omarsayed7/Deep-Emotion/blob/master/data_loaders.py
class EmotionDataset(Dataset):
    def __init__(self, csv_file, img_dir, datatype, transform):
        '''
        Pytorch Dataset class
        params:-
                 csv_file : the path of the csv file    (train, validation, test)
                 img_dir  : the directory of the images (train, validation, test)
                 datatype : data type for the dataset   (train, val, test)
                 transform: pytorch transformation over the data
        return :-
                 image, labels
        '''
        self.csv_file = pd.read_csv(csv_file)
        self.labels = self.csv_file['emotion']
        self.img_dir = img_dir
        self.datatype = datatype
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img = Image.open(self.img_dir + self.datatype + str(idx) + '.jpg').convert('RGB')
        labels = np.array(self.labels[idx])
        labels = torch.from_numpy(labels).long()

        if self.transform :
            img = self.transform(img)
        return img, labels

In [7]:
# tranformations
transformation= transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,], std=[0.5,])
])

In [8]:
# create datasets
train_dataset = EmotionDataset(
    csv_file=cfg.TRAIN_DS_PATH, 
    img_dir=cfg.TRAIN_IMG_DIR,
    datatype='train',
    transform=transformation
)

validation_dataset = EmotionDataset(
    csv_file=cfg.VAL_DS_PATH, 
    img_dir=cfg.VAL_IMG_DIR,
    datatype='val',
    transform = transformation
)

In [9]:
# create data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=cfg.BATCH_SIZE, 
    shuffle =cfg.SHUFFLE, 
    num_workers=cfg.NUM_WORKERS
)

val_loader = DataLoader(
    validation_dataset, 
    batch_size=cfg.BATCH_SIZE, 
    shuffle =cfg.SHUFFLE, 
    num_workers=cfg.NUM_WORKERS
)

# Helpers

In [10]:
def train_epoch(model, train_loader, criterion, optimizer, device, steps_print_loss=0):
    """Train the model for 1 epoch
    Args:
        model: nn.Module
        train_loader: train DataLoader
        criterion: callable loss function
        optimizer: pytorch optimizer
        device: torch.device
        steps_print_loss: loss will print out in every specified steps.
    Returns
    -------
    Tuple[Float, Float]
        average train loss and average train accuracy for current epoch
    """
    train_losses = []
    train_corrects = []
    model.train()

    # Iterate over data.
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # prediction
        outputs = model(inputs)

        # calculate loss
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # statistics
        train_losses.append(loss.item())
        train_corrects.append(torch.sum(preds == labels.data).item())
        
        if steps_print_loss and (batch_idx % steps_print_loss == 0):
            print(f"loss: {loss.item():>7f}  [{batch_idx:>5d}/{len(train_loader):>5d}]")
        
    train_loss = sum(train_losses)/len(train_losses)
    train_accuracy = sum(train_corrects)/len(train_loader.dataset)      

    return train_loss, train_accuracy


def val_epoch(model, val_loader, criterion, device):
    """Validate the model for 1 epoch
    Args:
        model: nn.Module
        val_loader: val DataLoader
        criterion: callable loss function
        device: torch.device
    Returns
    -------
    Tuple[Float, Float]
        average val loss and average val accuracy for current epoch
    """

    val_losses = []
    val_corrects = []
    model.eval()

    # Iterate over data
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # prediction
            outputs = model(inputs)

            # calculate loss
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # statistics
            val_losses.append(loss.item())
            val_corrects.append(torch.sum(preds == labels.data).item())
            
        val_loss = sum(val_losses)/len(val_losses)
        val_accuracy = sum(val_corrects)/len(val_loader.dataset)

    return val_loss, val_accuracy

# Training and Evaluating Model

In [11]:
# load pretrained model
model = models.mobilenet.mobilenet_v2(pretrained=True)

In [12]:
# freeze all layers
for param in model.parameters():
    param.requires_grad = False

num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, len(classes))

In [13]:
# transfer to cuda device if any
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using device:{device} for training")

Using device:cuda for training


In [14]:
# loss, optimizer and scheduler 
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=cfg.LR)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min')

In [15]:
# Start training and val loops

since = time.time()
for epoch in range(cfg.EPOCHS):
    print(f"Epoch {epoch+1}\n-------------------------------")
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, 500)
    print(f'Average TrainLoss: {train_loss:.4f} \tAverage TrainAcc: {train_acc:.4f}')
    
    # Evaluation
    print("Evaluation:")
    val_loss, val_acc = val_epoch(model, val_loader, criterion, device)
    print(f'Average ValLoss: {val_loss:.4f} \tAverage ValAcc: {val_acc:.4f}')
    
    # log metrics 
    if writer:
        writer.add_scalar("train_loss", train_loss, epoch)
        writer.add_scalar("train_accuracy", train_acc, epoch)
        writer.add_scalar("val_loss", val_loss, epoch)
        writer.add_scalar("val_accuracy", val_acc, epoch)
    
    # schedule lr
    scheduler.step(val_loss)
    print("\n")
        
time_elapsed = time.time() - since
print(f"Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s")

Epoch 1
-------------------------------
loss: 1.963119  [    0/ 3589]
loss: 1.736238  [  500/ 3589]
loss: 1.622846  [ 1000/ 3589]
loss: 1.515492  [ 1500/ 3589]
loss: 1.814029  [ 2000/ 3589]
loss: 1.745634  [ 2500/ 3589]
loss: 1.367832  [ 3000/ 3589]
loss: 1.790536  [ 3500/ 3589]
Average TrainLoss: 1.7197 	Average TrainAcc: 0.3010
Evaluation:
Average ValLoss: 1.6131 	Average ValAcc: 0.3664


Epoch 2
-------------------------------
loss: 1.336309  [    0/ 3589]
loss: 1.311972  [  500/ 3589]
loss: 1.804169  [ 1000/ 3589]
loss: 1.727910  [ 1500/ 3589]
loss: 1.772702  [ 2000/ 3589]
loss: 1.372732  [ 2500/ 3589]
loss: 1.378198  [ 3000/ 3589]
loss: 1.421382  [ 3500/ 3589]
Average TrainLoss: 1.6418 	Average TrainAcc: 0.3515
Evaluation:
Average ValLoss: 1.5710 	Average ValAcc: 0.3915


Epoch 3
-------------------------------
loss: 1.612230  [    0/ 3589]
loss: 1.671921  [  500/ 3589]
loss: 1.232558  [ 1000/ 3589]
loss: 1.323457  [ 1500/ 3589]
loss: 1.360630  [ 2000/ 3589]
loss: 1.248539  [ 2500

In [16]:
# Save the model
torch.save(model.state_dict(), cfg.MODEL_DIR + "model.pt")